In [ ]:
import fsspec

In [ ]:
fsspec.config.conf["s3"] = {
"default_cache_type": "blockcache",
"default_block_size": 2**22
}

In [ ]:
import earthaccess
import xarray as xr

In [ ]:
tspan = ("2024-08", '2024-08')
results = earthaccess.search_data(short_name="PACE_OCI_L3M_SFREFL", granule_name="*.MO.*.0p1deg.*", temporal=tspan)

In [ ]:
paths = earthaccess.open(results)

In [ ]:
dataset = xr.open_dataset(paths[0])

In [ ]:
dataset

In [ ]:
rgb = dataset["rhos"].sel({"wavelength": [645, 555, 440]}, method="nearest")

In [ ]:
plot = rgb.plot.imshow()

In [ ]:
import numpy as np
from PIL import Image, ImageEnhance

rgb
scale = 0.01
vmin = 0
vmax = 1.1
gamma = 1
contrast = 1.1
brightness = 1
sharpness = 1.1
saturation = 1

da = rgb.where(rgb > 0)
da = np.log(da / scale) / np.log(1 / scale)
da = da.clip(vmin, vmax)
da = (da - da.min()) / (da.max() - da.min())
da = da * gamma
da = da * 255
da = da.where(da.notnull(), 0).astype("uint8")
img = Image.fromarray(da.data)
enhancer = ImageEnhance.Contrast(img)
img = enhancer.enhance(contrast)
enhancer = ImageEnhance.Brightness(img)
img = enhancer.enhance(brightness)
enhancer = ImageEnhance.Sharpness(img)
img = enhancer.enhance(sharpness)
enhancer = ImageEnhance.Color(img)
img = enhancer.enhance(saturation)
rgb[:] = np.array(img) / 255

In [ ]:
plot = rgb.plot.imshow()